In [43]:
import os
from langchain_community.llms import LlamaCpp
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

In [47]:
# Step 1: Paths
DATA_DIR = "/Users/mansivarshney/Projects/Domain-Specific Chatbot/data"
INDEX_DIR = "/Users/mansivarshney/Projects/Domain-Specific Chatbot/rag_store"

# Step 2: Make sure rag_store exists
os.makedirs(INDEX_DIR, exist_ok=True)

# Step 3: Load all PDFs
docs = []
for file in os.listdir(DATA_DIR):
    if file.lower().endswith(".pdf"):
        loader = PyPDFLoader(os.path.join(DATA_DIR, file))
        docs.extend(loader.load())

# Step 4: If nothing loaded, stop
if not docs:
    raise ValueError("No documents found. Please check PDF files in ./data folder.")

# Step 5: Create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Step 6: Build FAISS index
vectorstore = FAISS.from_documents(docs, embeddings)

# Step 7: Save index
vectorstore.save_local(INDEX_DIR)

print(f"✅ Indexed {len(docs)} document chunks from {len(os.listdir(DATA_DIR))} PDF(s).")


✅ Indexed 1 document chunks from 3 PDF(s).


In [ ]:
# Step 8: Load FAISS index
vectorstore = FAISS.load_local(INDEX_DIR, embeddings, allow_dangerous_deserialization=True)

# Step 9: Create retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Step 10: Load local LLM (Llama.cpp)
llm = LlamaCpp(
    model_path="./models/llama-2-7b-chat.Q4_K_M.gguf",  # change to your model path
    temperature=0.1,
    max_tokens=512,
    n_ctx=2048,
    verbose=False
)

# Step 11: Create RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

# Step 12: Ask questions
while True:
    query = input("\nAsk a question (or type 'exit'): ")
    if query.lower() == "exit":
        break
    answer = qa_chain.invoke({"query": query})
    print("\nAnswer:", answer)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ./models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32       


Ask a question (or type 'exit'):  What are the common symptoms of late blight in tomatoes?


/var/folders/pc/0cgfs26s25q92hz0m02252p40000gn/T/ipykernel_14804/2350665892.py:27: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(query)



Answer:  Late blight is a bacterial disease that affects tomato plants and can cause a range of symptoms. The most common symptoms of late blight include:
1. Yellowing or browning of leaves, often accompanied by a distinctive "target" or "bullseye" pattern on the leaf surface. This is caused by the bacteria producing a toxin that kills cells in the leaf tissue.
2. Premature defoliation, as the bacteria can cause the leaves to drop off the plant before their normal time.
3. Distortion or deformation of leaves and stems, which can make them appear "cupped" or "twisted".
4. Softening or rotting of fruit, particularly on the underside, which can lead to premature drop.
5. Stem lesions or cankers, which are open sores on the stem that can ooze a sticky, bacterial slime.
6. Root rot, which can cause the plant's roots to decay and become waterlogged.
7. Reduced plant growth and productivity, as late blight can weaken the plant and reduce its ability to produce fruit.
It is important to note 


Ask a question (or type 'exit'):  Hi, I need advice on tomato plant care?



Answer:  I'm not able to provide advice on tomato plant care as it is not within my area of expertise. I'm just an AI and do not have personal experience or knowledge in gardening or agriculture. My training data does not include information on tomato plant care, and I cannot provide accurate or helpful information on this topic.

Please let me know if you have any other questions or topics you would like to discuss!
